# 02 : GPU Check

This is a simple test to see if the GPU is available and working correctly.

- https://stackoverflow.com/questions/76581229/is-it-possible-to-check-if-gpu-is-available-without-using-deep-learning-packages
- https://docs.mlrun.org/en/v1.7.2/runtimes/configuring-job-resources.html
- https://docs.k3s.io/advanced#nvidia-container-runtime

In [1]:
import mlrun

In [2]:
# Show the API server URL
mlrun.get_run_db()

HTTPRunDB('http://dragon:30070')

In [3]:
# Set the base project name
project_name = "mlrun-demo"

# Initialize the MLRun project object
project = mlrun.get_or_create_project(
    name=project_name, 
    context="./",
    user_project=True)

# Display the current project name
project_name = project.metadata.name
print(f'Full project name: {project_name}')

> 2025-07-15 14:41:25,383 [info] Loading project from path: {"path":"./","project_name":"mlrun-demo","user_project":true}
> 2025-07-15 14:41:25,452 [info] Project loaded successfully: {"path":"./","project_name":"mlrun-demo-johannes","stored_in_db":true}
Full project name: mlrun-demo-johannes


## Get GPU Function

In [4]:
%%writefile 02_get_gpu_info.py

import GPUtil

def get_gpu_info(context):    
    gpus = GPUtil.getGPUs()
    gpu_info = []
    for gpu in gpus:
        gpu_info.append({
            'id': gpu.id,
            'name': gpu.name,
            'load': gpu.load,
            'memory_total': gpu.memoryTotal,
            'memory_free': gpu.memoryFree,
            'memory_used': gpu.memoryUsed,
        })

    context.logger.info(f"GPU Info: {gpu_info}")
    return gpu_info

Overwriting 02_get_gpu_info.py


## ML Run Function

In [5]:
fn_gpu_check = project.set_function(
    func="02_get_gpu_info.py",
    name="gpu-check",
    tag="latest",
    kind="job",
    image="mlrun/mlrun-gpu",
    handler="get_gpu_info",
    requirements=["GPUtil==1.4.0"])

# Then set the GPU resources on the function's spec
fn_gpu_check.with_requests(mem="1G", cpu=1)  # lower bound
fn_gpu_check.with_limits(mem="2G", cpu=2, gpus=1)  # upper bound
# fn_gpu_check.spec.resources = {
#     "limits": {"nvidia.com/gpu": 1},
#     "requests": {"nvidia.com/gpu": 1}
# }

In [ ]:
# run the function locally
fn_gpu_check.run(
    local=False,
    handler="get_gpu_info",
    auto_build=True
)

> 2025-07-15 14:41:25,536 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe
> 2025-07-15 14:41:25,553 [error] error getting build status: details: MLRunNotFoundError('Function tag not found mlrun-demo-johannes/gpu-check:latest'), caused by: 404 Client Error: Not Found for url: http://dragon:30070/api/v1/build/status?name=gpu-check&project=mlrun-demo-johannes&tag=latest&logs=no&offset=0&last_log_timestamp=0.0&verbose=no
> 2025-07-15 14:41:25,553 [info] Function is not deployed and auto_build flag is set, starting deploy...
> 2025-07-15 14:41:25,561 [error] error getting build status: details: MLRunNotFoundError('Function tag not found mlrun-demo-johannes/gpu-check:latest'), caused by: 404 Client Error: Not Found for url: http://dragon:30070/api/v1/build/status?name=gpu-check&project=mlrun-demo-johannes&tag=latest&logs=no&offset=0&last_log_timestamp=0.0&verbose=no
> 2025-07-15 14:41:25,562 [warning] Even t